In [17]:
# Import modules
import pandas as pd
import cleaning
import utils

In [18]:
# open elections.xlsx
elections = pd.read_excel('elections.xlsx')
# open elections_with_geocodes.xlsx
geocodes = pd.read_excel('elections_with_geocodes.xlsx')
# open elections_with_tracts.xlsx
tracts = pd.read_excel('elections_with_tracts.xlsx')

groups = ['DP02', 'DP03', 'DP05']
key = 'ad8851f3bf6aaf76923ec4119b6f714cdfaa87d9'

In [19]:
tracts_dict = {'geocode': [], 'year': []}

# for each row in elections
for index, row in elections.iterrows():
    # if District # is not nan
    if not pd.isna(row['District #']):
        # get row in tracts where filename is the same
        tract_row = tracts[tracts['filename'] == row['filename']]
        # make sure is not nan
        if not pd.isna(tract_row['tracts'].iloc[0]):
            new_tracts = tract_row['tracts'].iloc[0].split(', ')
            # add to tracts_df
            for new_tract in new_tracts:
                tracts_dict['geocode'].append(new_tract)
                tracts_dict['year'].append(row['year'])
            
# create tracts_df
tracts_df = pd.DataFrame(tracts_dict)

# remove duplicates
tracts_df = tracts_df.drop_duplicates()

# reset index on tracts_df
tracts_df = tracts_df.reset_index(drop=True)

In [20]:
acs_data = []

In [22]:
# for each row in tracts_df
for index, row in tracts_df.iterrows():
    if index < len(acs_data):
        continue

    if index % 10 == 0:
        print(f"{index} of {len(tracts_df)}")

    geocode = row['geocode'].split('_')
    state = geocode[0].zfill(2)
    county = geocode[1].zfill(3)
    tract = geocode[2].zfill(6)
    year = row['year']
    
    for_param = f'tract:{tract}&in=state:{state}+county:{county}'
    data_dict = utils.query(year, groups, for_param, key)
    data_dict['geocode'] = row['geocode']
    data_dict['year'] = year

    acs_data.append(data_dict)

4580 of 6410
4590 of 6410
4600 of 6410
4610 of 6410
4620 of 6410
4630 of 6410
4640 of 6410
4650 of 6410
4660 of 6410
4670 of 6410
4680 of 6410
4690 of 6410
4700 of 6410
4710 of 6410
4720 of 6410
4730 of 6410
4740 of 6410
4750 of 6410
4760 of 6410
4770 of 6410
4780 of 6410
4790 of 6410
4800 of 6410
4810 of 6410
4820 of 6410
4830 of 6410
4840 of 6410
4850 of 6410
4860 of 6410
4870 of 6410
4880 of 6410
4890 of 6410
4900 of 6410
4910 of 6410
4920 of 6410
4930 of 6410
4940 of 6410
4950 of 6410
4960 of 6410
4970 of 6410
4980 of 6410
4990 of 6410
5000 of 6410
5010 of 6410
5020 of 6410
5030 of 6410
5040 of 6410
5050 of 6410
5060 of 6410
5070 of 6410
5080 of 6410
5090 of 6410
5100 of 6410
5110 of 6410
5120 of 6410
5130 of 6410
5140 of 6410
5150 of 6410
5160 of 6410
5170 of 6410
5180 of 6410
5190 of 6410
5200 of 6410
5210 of 6410
5220 of 6410
5230 of 6410
5240 of 6410
5250 of 6410
5260 of 6410
5270 of 6410
5280 of 6410
5290 of 6410
5300 of 6410
5310 of 6410
5320 of 6410
5330 of 6410
5340 of 6410

In [23]:
df = cleaning.dicts_to_df(acs_data)

# order so that geocode and year are first
df = df[['geocode', 'year'] + [col for col in df.columns if col not in ['geocode', 'year']]]

# save as csv
df.to_csv('tract_acs_data.csv', index=False)